### Run following code only for Google Colab

In [1]:
from google.colab import drive
drive.mount('/content/drive')
!pip install flask-ngrok
!pip install torch==1.7.1
!pip install transformers==3.5.1

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Code to connect to mysql connector

In [2]:
# import mysql.connector
# mydb = mysql.connector.connect(
#   host="localhost",
#   user="yourusername",
#   password="yourpassword"
# )

# print(mydb)

In [54]:
import os
os.chdir('/content/drive/MyDrive/Work/deployment_distilbert')
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import re
import unicodedata
import nltk
from nltk.corpus import stopwords
import keras
from tqdm import tqdm
import pickle
from keras.models import Model
import keras.backend as K
from sklearn.metrics import confusion_matrix,f1_score,classification_report
import matplotlib.pyplot as plt
from keras.callbacks import ModelCheckpoint
import itertools
from keras.models import load_model
from sklearn.utils import shuffle
import torch
from transformers import DistilBertTokenizer, TFDistilBertModel, DistilBertConfig
from transformers import TFDistilBertForSequenceClassification
import torch.nn.functional as nnf
import time
import torch.nn as nn
import pandas as pd
DEVICE = "cuda"
max_len = 64
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
EPOCHS = 10 
BERT_PATH = "distilbert-base-uncased"
MODEL_PATH = "DistilBert_modelh.h5"
TRAINING_FILE = "train_iris_fullcleaned.xlsx"
tokenizer = DistilBertTokenizer.from_pretrained(BERT_PATH, do_lower_case=True)
df=pd.read_excel(TRAINING_FILE)
num_classes=len(df.intent1.unique())
d= dict([(i,a) for i, a in zip(df.intent1, df.intent)])

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5,epsilon=1e-08)
MODEL = TFDistilBertForSequenceClassification.from_pretrained(BERT_PATH,num_labels=num_classes)
MODEL.compile(loss=loss,optimizer=optimizer, metrics=[metric])
MODEL.load_weights(MODEL_PATH)

def pred(text,tokenizer,MODEL):
  maxindx,predictions,maxindx1,maxindx2,maxindx3,maxindx4,predictions1,predictions2,predictions3,predictions4=[],[],[],[],[],[],[],[],[],[]
  top_p1,top_p2,top_p3,top_class1,top_class2,top_class3=[],[],[],[],[],[]

  DistilBert_inp=tokenizer.encode_plus(text,add_special_tokens = True,max_length =64,pad_to_max_length = True,return_attention_mask = True)
  input_ids=DistilBert_inp['input_ids']
  attention_masks=DistilBert_inp['attention_mask']
  input_ids=np.asarray(input_ids).reshape(-1,64)
  attention_masks=np.array(attention_masks).reshape(-1,64)
  outputs = MODEL.predict([input_ids,attention_masks],batch_size=32)
  maxidx = outputs[0].argmax(axis=1)
  maxindx.append(int(maxidx))

  predictions.append(d[int(maxidx)])
  prob = nnf.softmax(torch.tensor(outputs[0]),dim=1)
  top_p, top_class = prob.topk(3, dim = 1)

  top_p1.append(float(top_p[0][0]))
  top_p2.append(float(top_p[0][1]))
  top_p3.append(float(top_p[0][2]))
  top_class1.append(int(top_class[0][0]))
  top_class2.append(int(top_class[0][1]))
  top_class3.append(int(top_class[0][2]))

  predictions1.append(d[int(top_class[0][0])])
  predictions2.append(d[int(top_class[0][1])])
  predictions3.append(d[int(top_class[0][2])])
  sent_csv={}
  sent_csv["pred_idx"]=maxindx[0]
  sent_csv["pred_values"]=predictions[0]
  sent_csv["top_p1"]=top_p1[0]
  sent_csv["top_p2"]=top_p2[0]
  sent_csv["top_p3"]=top_p3[0]
  sent_csv["top_class1"]=top_class1[0]
  sent_csv["top_class2"]=top_class2[0]
  sent_csv["top_class3"]=top_class3[0]
  sent_csv["predictions1"]=predictions1[0]
  sent_csv["predictions2"]=predictions2[0]
  sent_csv["predictions3"]=predictions3[0]
  return sent_csv

Writing utils.py


In [56]:
from flask_ngrok import run_with_ngrok
from flask import Flask
from string import Template
from flask import Flask, render_template, request

app = Flask(__name__)
run_with_ngrok(app)

@app.route('/')
def home():
	return render_template('home.html')

@app.route('/predict',methods=['POST'])
def predict():
  if request.method=='POST':
    message = request.form['message']
    p=pred(message,tokenizer,MODEL)  
  return render_template('result.html',prediction = p)

app.run()

Overwriting app.py
